## This notebook focuses on two main objectives:

### 1. Accelerating Decision Making: Building a new feature to enhance artist recommendation.
### 2. Impact of NLP Entity Resolution: Demonstrating improvements in recommendations through named entity resolution for semantically-equivalent names.

In [1]:
import os
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt

In [2]:
data_dir = f"{os.getcwd()}/data"

In [3]:
df = pd.read_csv(f"{data_dir}/spotify_dataset.csv",
                    skiprows=1,
                    names=['user_id', 'artistname', 'trackname', 'playlistname'],
                    on_bad_lines='skip')
df = df.sample(n=10000, random_state=42)
print(df.info())
df.head(5)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10000 entries, 7578551 to 10128040
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   user_id       10000 non-null  object
 1   artistname    9970 non-null   object
 2   trackname     10000 non-null  object
 3   playlistname  9999 non-null   object
dtypes: object(4)
memory usage: 390.6+ KB
None


user_id         artistname  \
7578551   5c2b840d66d02ac7aa6a069eddd17f48        Spice Girls   
911963    6da651f45aac18e5da52433aa3c38947       Flying Lotus   
8049506   20350023c94899ce25cd1a86ea385853      Kristin Hersh   
11247838  f9809d1afbab1b0aaa0c2f678a90752a  Rotary Connection   
6160700   db9646b77f3a677040b9140489da16f1           Bastille   

                                        trackname             playlistname  
7578551                      Wannabe - Radio Edit  Lol HiGh Sk00l MeMorIeZ  
911963    Descent Into Madness (feat. Thundercat)             Best of 2014  
8049506                                Your Ghost  Ladies. All the ladies.  
11247838                                This Town             To listen to  
6160700                                   Pompeii                  Starred

In [4]:
df.describe()

user_id          artistname trackname  \
count                              10000                9970     10000   
unique                              5019                5696      9347   
top     4398de6902abde3351347b048fcdc287  The Rolling Stones     Mercy   
freq                                 210                  28         6   

       playlistname  
count          9999  
unique         6640  
top         Starred  
freq           1017

In [5]:
# null values in dataframe
df.isnull().sum()

user_id          0
artistname      30
trackname        0
playlistname     1
dtype: int64

In [6]:
df[df['artistname'].isnull()]

user_id artistname  \
10582452  9b0cb216778138cc481d17dee58a0319        NaN   
10366650  93000d9cad22c61869db84fb96c8b5c9        NaN   
8186225   c8e0b3f55f4ecd3c0c7947a18e7472d8        NaN   
3642727   74bc80e9a16ccd15a85f8d4d871000cf        NaN   
8516423   67bb38768667210fe57b985c96f5fa4c        NaN   
7512326   c738ccdf6921c156a815e9fa9f8b9dd9        NaN   
7827501   2801dbeaa27d88758fc964e70b64f7bc        NaN   
6767682   c6f591538735f1c7c25964edfcbe81b8        NaN   
1447006   cba662ee7038612852220efd5730e616        NaN   
5036191   14d4bc2cb64c89d7180421e93afc271a        NaN   
1099753   c84332c8d946677b2df2d478c215518e        NaN   
10582259  9b0cb216778138cc481d17dee58a0319        NaN   
8652298   dc22b0897ce2674dffd4e6b4fab314ad        NaN   
8297475   31784dd58f461135fe60a5eea77b8f8b        NaN   
6935775   a32fcbb97e7315f4b142ceb834ba4894        NaN   
8521191   67bb38768667210fe57b985c96f5fa4c        NaN   
9476997   7ffae624f8f096dd429b1f7e78efb32b        NaN   
4869476   1a3911cef9866d4d6c86271f7d7172a3        NaN   
2717551   3f27634a1b38377b8d7f806f653dccbe        NaN   
6966440   3d266f726ca3f8a9cd808d6e94975846        NaN   
341751    1de451174570ab0e4c20da61b54bed98        NaN   
8183746   c8e0b3f55f4ecd3c0c7947a18e7472d8        NaN   
8183538   c8e0b3f55f4ecd3c0c7947a18e7472d8        NaN   
1906444   40ef782b7710a0614d27a077644f3d6a        NaN   
292024    2a60752bad128b0ab322e8f3048df13d        NaN   
8184679   c8e0b3f55f4ecd3c0c7947a18e7472d8        NaN   
4521238   437dbe677279b2d45e6d2e1070652885        NaN   
9211519   492d6f1cce123ec653548bc9374373d2        NaN   
3249568   25deb8fd8a3c7b2883de9701860ec395        NaN   
985899    1110040a3114b27b564547255bbce924        NaN   

                                                  trackname  \
10582452  avicii vs nicky romero - i could be the one (n...   
10366650        Animal Crossing New Leaf - 12 PM (Extended)   
8186225   16 - Joni Mitchell - Both Sides Now - www.torr...   
3642727                                          Gee - SNSD   
8516423         Ben Pearce - What I Might Do (Kilter Remix)   
7512326                                             Spor 08   
7827501   Gonzales - Z - 13 - Let's Groove Again ft. Lou...   
6767682                         Dio - Holy Diver - drumless   
1447006                                    Kaleido Star Op2   
5036191                                     16 ANOCHECE.mp3   
1099753   Roscoe Dash - Good Good Night Lyrics (Official...   
10582259                          Dennis Ferrer Hey Hey.mp3   
8652298                  With a little help from my friends   
8297475               Sigma - Nobody To Love (Fanfar Remix)   
6935775                    Estopa y Muchachito - TU CALORRO   
8521191               Robin S - Show Me Love (Siloet Remix)   
9476997                           Miguel N√°ndez - Mares.mp3   
4869476             antonio vega y amaral - como hablar.mp3   
2717551                                    34. il cuore.mp3   
6966440                                            Pista 05   
341751                         los piratas-relax-inerte.mp3   
8183746                                           06 Wheels   
8183538                                 06 - I Can't Let Go   
1906444        09 Crazy For You (Idaho's Smokey Addict Mix)   
292024         Erase you (Rmx Marcos Rodriguez & Xavi Beat)   
8184679                                  09 - Train Of Love   
4521238   Santogold - You'll Find A Way (Switch & Graeme...   
9211519                                  15 - Cuerpo muerto   
3249568   edward maya feat alicia - stereo love (origina...   
985899                           09 Left Hand Suzuki Method   

                           playlistname  
10582452                        Bautizo  
10366650                Animal Crossing  
8186225                             MMM  
3642727                            Osu!  
8516423            1 wintah starred 14'  
7512326                          BCN 06  
7827501   Gregory 

In [7]:
# drop nulls
df.dropna(inplace=True)

In [8]:
# duplicates
df[df.duplicated()]

Empty DataFrame
Columns: [user_id, artistname, trackname, playlistname]
Index: []

## Exploratory Data Analysis

#### This dataset doesn't provide repeat of track/s, but merely tracks that have been played, and metadata like playlist and artistname the track belongs to! 
### Opportunity to learn more about user-artist preference. 

In [9]:
# artist frequently played by user
df.groupby(['user_id', 'artistname'], as_index=False
          )['trackname'].count().sort_values(by=['trackname'], ascending=False)

user_id              artistname  trackname
5566  8c3e1191cb28f7c57bbf35c1e2ad1bf1         Aretha Franklin          5
1422  26b51e580277e131f87e4c7ee4c0887a  Vitamin String Quartet          5
9620  fa6ec384798989855ad898e6b8c68c3b           John Williams          4
2471  414050deadb38aafd8d4ad22ca634055  Vitamin String Quartet          4
6422  a251c8204b5bc479dc5c83c15b7cc929      The Rolling Stones          3
...                                ...                     ...        ...
3293  53f433aeae7d61e5e977a61342a92740          Duke Ellington          1
3294  54174df459c3780a3251a9ce6506db09               Sam Smith          1
3295  541a9ceaaf98e45954894532ec57f30b        The Decemberists          1
3296  543d13a267c4f91c511fdaf97a13f86b                  Grises          1
9835  fff616055993498d6127f3f467cf9f2b              Cher Lloyd          1

[9836 rows x 3 columns]

In [10]:
# track making into different playlist
df.groupby(['artistname', "trackname"], as_index=False
          )['playlistname'].nunique().sort_values(by=['playlistname'], ascending=False)

artistname                            trackname  \
2641                   Eminem   Lose Yourself - Soundtrack Version   
7030                  Rihanna                        We Found Love   
615                    Avicii                          Hey Brother   
7076             Robin Thicke                        Blurred Lines   
5735             Modest Mouse                             Float On   
...                       ...                                  ...   
3257  Good Knight Productions  Sonic Adventure 2 - Way to the Base   
3258           Goran Bregovic                               RU≈ΩICA   
3259         Gordon Lightfoot                            Beautiful   
3260                  Gorguts                     Le toit du monde   
9625                      Á¥îÈü≥Ê®Ç                                   ÂæíÁÑ∂   

      playlistname  
2641             4  
7030             4  
615              4  
7076             4  
5735             4  
...            ...  
3257             1  
3258             1  
3259             1  
3260             1  
9625             1  

[9626 rows x 3 columns]

In [11]:
# count of artist by playlist
df.groupby(['playlistname'], as_index=False
          )['artistname'].nunique().sort_values(by=['artistname'], ascending=False)

playlistname  artistname
4739                                      Starred         868
2955                             Liked from Radio         130
1874                        Favoritas de la radio          26
201                                          2013          23
1198                                    Christmas          20
...                                           ...         ...
2363               How To Dress Well ‚Äì Total Loss           1
2362                  How To Disappear Completely           1
2361  How I Met Your Mother (music from the show)           1
2360                            House on SOUNDROP           1
6622                                            üöÑ           1

[6623 rows x 2 columns]

## Preprocessing artist name 

### text normalization

In [12]:
import re
# Function to normalize text
def normalize(text):
    text = text.lower() 
    text = re.sub(r'\s+', ' ', text)  # Replace multiple spaces with a single space
    text = re.sub(r"[.'‚Äô]", '', text)  # Remove punctuation
    text = re.sub(r'^(the|a)\s+', '', text) # Remove the, a from start of the text
    return text

# Normalize all artist names
df['normalized_artistname'] = df['artistname'].apply(normalize)
df

user_id            artistname  \
7578551   5c2b840d66d02ac7aa6a069eddd17f48           Spice Girls   
911963    6da651f45aac18e5da52433aa3c38947          Flying Lotus   
8049506   20350023c94899ce25cd1a86ea385853         Kristin Hersh   
11247838  f9809d1afbab1b0aaa0c2f678a90752a     Rotary Connection   
6160700   db9646b77f3a677040b9140489da16f1              Bastille   
...                                    ...                   ...   
4892779   529f12438bfbfc2c67a15cb1bc915e26                Agaric   
9544566   ce1ac1277e6abbe4b232f8e92b83fbdf                  2Pac   
2682421   4cecda8e0ebdcc4c664aa3598a35c43b            James Curd   
11586113  658159ed7b84bf407b1edd7bf7ce30c8               DJ Jump   
10128040  f5e5c54d5dd36b27d28619b0b2aa3b83  Tensnake feat. Fiora   

                                        trackname  \
7578551                      Wannabe - Radio Edit   
911963    Descent Into Madness (feat. Thundercat)   
8049506                                Your Ghost   
11247838                                This Town   
6160700                                   Pompeii   
...                                           ...   
4892779   No Way I Know I Feel - Axel Boman Remix   
9544566                       Got My Mind Made Up   
2682421                                  A Friend   
11586113                                On My Own   
10128040                        See Right Through   

                                               playlistname  \
7578551                             Lol HiGh Sk00l MeMorIeZ   
911963                                         Best of 2014   
8049506                             Ladies. All the ladies.   
11247838                                       To listen to   
6160700                                             Starred   
...                                                     ...   
4892779                                     Ovum Recordings   
9544566                                             truckin   
2682421     NuDisco/ObscureDisco/House/DeepHouse/IndieDance   
11586113              Various Artists ‚Äì 101 Baladas del Pop   
10128040  Ben Pearce - Pure Deep House - The Very Best o...   

         normalized_artistname  
7578551            spice girls  
911963            flying lotus  
8049506          kristin hersh  
11247838     rotary connection  
6160700               bastille  
...                        ...  
4892779                 agaric  
9544566                   2pac  
2682421             james curd  
11586113               dj jump  
10128040   tensnake feat fiora  

[9969 rows x 5 columns]

### Named Entity Resolution (NER) for semantically-equivalent names 

In [13]:
import difflib
def cleanup_dict(equivalent_names):
    new_equivalent_names = {}
    repeatative_names = []
    for k, v in equivalent_names.items():
        if k in repeatative_names:
            continue
        new_equivalent_names[k] = v
        repeatative_names.extend(v)
    return new_equivalent_names

def find_equivalent_names(list_artists):
    equivalent_names = {}
    for artist in list_artists:
        matches = difflib.get_close_matches(artist, list_artists, 
                                            n=5, cutoff=0.7
                                           ) #manage cutoff to get closest match based on requirement
        if len(matches) > 1:
            equivalent_names[artist] = matches
    return equivalent_names

equivalent_artist_names = cleanup_dict(
                            find_equivalent_names([str(n) for n in df['normalized_artistname'].unique()])
                        )

# print first 30 items from dictionary
i = 0
for k, v in equivalent_artist_names.items():
    print(k, v)
    i += 1
    if i == 30:
        break

spice girls ['spice girls', 'cheap girls']
kristin hersh ['kristin hersh', 'kristin chenoweth']
bastille ['bastille', 'stills', 'ballet', 'bagatelle']
skillet ['skillet', 'skrillex', 'killers']
since october ['since october', 'spin doctors', 'blue october']
linkin park ['linkin park', 'clinton sparks']
blur ['blur', 'blu', 'blue']
16volt ['16volt', '16 volt']
crystal castles ['crystal castles', 'crystal stilts', 'crystal fighters']
hives ['hives', 'givers']
andrew bird ['andrew bird', 'andrew ripp', 'andrew gold', 'new breed']
bongwater ['bongwater', 'ben walther']
bravery ['bravery', 'ralvero']
train ['train', 'trini', 'travis', 't-pain', 't pain']
snow patrol ['snow patrol', 'soul patrol']
black books ['black books', 'bad books', 'black rose', 'black kids', 'black keys']
future islands ['future islands', 'futureheads']
cars ['cars', 'mc lars', 'caesars']
soundgarden ['soundgarden', 'sound nomaden']
ryan adams ['ryan adams', 'bryan adams']
rancid ['rancid', 'acid', 'ramoncin']
dash be

## This self-identified list of semantically-equivalent artist names is intended for merging into single named entities. 

## With input from subject matter experts like content editors, the list can become more precise and accurate, enhancing model performance and reliability

In [14]:
# Identified artistnames that are semantically equivalent to be merged into a single named entity.
# With expert's knowledge we can create an accurate and 
semantically_equivalent_artistnames = {'16 volt':'16volt',
 '30 seconds to mars':'thirty seconds to mars',
 'b dylan': 'bob dylan',
 'dave matthews band':'dave matthews',
 'bjork': 'bj√∂rk',
 'jay z': 'jay-z',
 'paul mccartney & wings':'paul mccartney',
  'original motion picture soundtrack':'motion city soundtrack',
 'macklemore & ryan lewis ft mary lambert': 'macklemore & ryan lewis',
 'imogen heap':'imagen heap',
 'pat metheny group':'pat metheny',
 'guns and roses':'guns n roses', 
 'belle & sebastian':'belle and sebastian',
 'sylvan':'sylvan esso',
 'william fitzsimmons & michael flynn':'william fitzsimmons',
 'drumsound & bassline smith': 'drumsound & bassline',
 'oscar de leon' : 'oscar dle√≥n',
 'london symphony orchestra': 'london philharmonic orchestra',
  'relaxing piano music': 'relaxing piano music consort', 
 'florence + the machine': 'florence and the machine',
 'hans zimmer & james newton howard': 'james newton howard',
 'led zepagain': 'led zeppelin',
 'coheed & cambria': 'coheed and cambria',
 'beyonce ft jay z' : 'beyonce',
 'beyonce feat kanye west' : 'beyonce',
 'simon & garfunkel': 'simon and garfunkel', 
 'oscar peterson trio': 'oscar peterson',
 'dave brubeck':'dave brubeck quartet',
 'roc√≠o d√∫rcal': 'rocio durcal',  
 'joaquin sabina':'joaqu√≠n sabina',
 'ray barretto': 'ray barreto',
 'kenny garrett quintet': 'kenny garrett',
 'elvis costello and the brodsky quartet' : 'elvis costello' ,
 'elvis costello & the imposters' : 'elvis costello',
 'elvis costello & the attractions' : 'elvis costello' ,
 'various artists - abba tribute': 'various artists - the eagles tribute',
}

In [52]:
# Replace artistname to it's semantically equivalent. Reducing noise.
df['semantical_equivalent_normalized_artistname'] = df['normalized_artistname'].apply(
    lambda x: semantically_equivalent_artistnames[x] if x in semantically_equivalent_artistnames else x)

# print
df[df['semantical_equivalent_normalized_artistname'].isin(semantically_equivalent_artistnames.values())
  ].groupby(['semantical_equivalent_normalized_artistname','normalized_artistname']).count().head(20)

user_id  \
semantical_equivalent_normalized_artistname normalized_artistname                       
16volt                                      16 volt                                 1   
                                            16volt                                  1   
belle and sebastian                         belle & sebastian                      11   
                                            belle and sebastian                     2   
beyonce                                     beyonce feat kanye west                 1   
                                            beyonce ft jay z                        1   
bj√∂rk                                       bjork                                   1   
                                            bj√∂rk                                   2   
bob dylan                                   b dylan                                 1   
                                            bob dylan                              18   
coheed and cambria                          coheed & cambria                        2   
                                            coheed and cambria                      1   
dave brubeck quartet                        dave brubeck                            2   
                                            dave brubeck quartet                    1   
dave matthews                               dave matthews                           2   
                                            dave matthews band                     17   
drumsound & bassline                        drumsound & bassline                    1   
                                            drumsound & bassline smith              1   
elvis costello                              elvis costello                          4   
                                            elvis costello & the attractions        1   

                                                                              artistname  \
semantical_equivalent_normalized_artistname normalized_artistname                          
16volt                                      16 volt                                    1   
                                            16volt                                     1   
belle and sebastian                         belle & sebastian                         11   
                                            belle and sebastian                        2   
beyonce                                     beyonce feat kanye west                    1   
                                            beyonce ft jay z                           1   
bj√∂rk                                       bjork                                      1   
                                            bj√∂rk                                      2   
bob dylan                                   b dylan                                    1   
                                            bob dylan                                 18   
coheed and cambria                          coheed & cambria                           2   
                                            coheed and cambria                         1   
dave brubeck quartet                        dave brubeck                               2   
                                            dave brubeck quartet                       1   
dave matthews                               dave matthews                              2   
                                            dave matthews band                        17   
drumsound & bassline                        drumsound & bassline                       1   
                                            drumsound & bassline smith                 1   
elvis costello                              elvis costello                             4   
                                            elvis costello & the attractions           1   

                                                                              trackname  \

In [54]:
# entity list identified as semantically equivalent
artistname_semtanically_equi = list(df_artistnames_semantically_equ.groupby('semantical_equivalent_normalized_artistname'
                                                                           )['normalized_artistname'].first().reset_index()['normalized_artistname'])
artistname_entity = list(df_artistnames_semantically_equ['semantical_equivalent_normalized_artistname'].unique())



# Count the occurrences of each entity before and after NER
original_counts = df['normalized_artistname'][df['normalized_artistname'].isin(artistname_semtanically_equi)
                                ].value_counts().reindex(artistname_semtanically_equi)

after_ner_counts = df['semantical_equivalent_normalized_artistname'][df['semantical_equivalent_normalized_artistname'].isin(artistname_entity)
                                ].value_counts().reindex(artistname_entity)



r = np.arange(len(artistname_entity)) 
width = 0.3

# Create subplots
fig, axes = plt.subplots(1, 1, figsize=(14, 7))
plt.bar(r , original_counts.tolist(), color='b', width = width, label='Original Data')
plt.bar(r + width, after_ner_counts.tolist(), color='g', width = width, label='After Named Entity Resolution')

plt.xticks(r + width/2, artistname_entity, rotation=90)
plt.xlabel('ArtistNames')
plt.ylabel('Count')
plt.title('Entity Resolution reduces noise')
plt.legend() 
plt.show()

## User-Artist Interaction Matrix

In [57]:
# named entity resolution - user artist matrix 
user_artist_ner_matrix = df.groupby(['user_id', 'semantical_equivalent_normalized_artistname']
                               )['semantical_equivalent_normalized_artistname'].count().unstack().reset_index().fillna(0)
user_artist_ner_matrix.set_index('user_id', inplace=True)

# user artist matrix 
user_artist_matrix = df.groupby(['user_id', 'normalized_artistname']
                               )['normalized_artistname'].count().unstack().reset_index().fillna(0)
user_artist_matrix.set_index('user_id', inplace=True)                             

In [58]:
print(f"Applying Named Entity Resolution reducing dimensionality {len(user_artist_matrix.columns)}\
 to {len(user_artist_ner_matrix.columns)} ")

Applying Named Entity Resolution reducing dimensionality 5665 to 5630 


##### Techniques like K-means, DBScan will not yield meaningful result with sparse feature unless we further apply dimensionality reduction.
## User-based collaborative filtering

In [59]:
from sklearn.metrics.pairwise import cosine_similarity

similar_users_dict = {}

def user_cosine_similarity(user_artist_matrix):
    
    # Compute the cosine similarity between users
    user_similarity = cosine_similarity(user_artist_matrix)
    # Create a DataFrame for user similarity
    df_user_similarity = pd.DataFrame(user_similarity, index=user_artist_matrix.index, columns=user_artist_matrix.index)
    return df_user_similarity

def recommend_top_artists(user_id, df_user_similarity, user_artist_matrix, similar_users = 5, top_artist = 10):
    def get_top_n_similar_users(user_id, n):
        list_similar_users = df_user_similarity[user_id].sort_values(ascending=False).head(n+1).index.tolist()
        if user_id in list_similar_users:
            list_similar_users.remove(user_id)
            
        return list_similar_users
    
    list_similar_users = get_top_n_similar_users(user_id, similar_users)
    artist_scores = user_artist_matrix.loc[list_similar_users].mean(axis=0)
    top_artists = artist_scores.sort_index().sort_values(ascending=False).head(top_artist).index.tolist()
    return list_similar_users, top_artists


user_id = '0025022960e5f0d7d01af5d840014594'
matrix = user_artist_matrix
list_similar_users, top_artists = recommend_top_artists(user_id, user_cosine_similarity(matrix), matrix,
                                    similar_users = 5, top_artist = 3
                                   )
print(f"Top 10 artist recommendations for user {user_id}: {top_artists}")

Top 10 artist recommendations for user 0025022960e5f0d7d01af5d840014594: ['robbie williams', 'english chamber orchestra, conducted by jeffrey tate', 'jethro tull']


### Example : Analyze difference in before recommendation vs NER recommandation

In [65]:
user_id = '5c2b840d66d02ac7aa6a069eddd17f48'
top_artist = 5
similar_users = 5

matrix = user_artist_matrix
list_similar_users, top_artists = recommend_top_artists(user_id, user_cosine_similarity(matrix), matrix,
                                    similar_users = similar_users, top_artist = top_artist
                                   )
print(f"Top {top_artist} artist recommendations for user {user_id} : \n {top_artists} \n")
print(f"Here are the {similar_users} similar users \n \t {list_similar_users}")
for u in list_similar_users:
    print(f"\t Artist frequented by {u}: \n \
            {user_artist_matrix.loc[u].sort_index().sort_values(ascending=False).head(top_artist).index.tolist()} ")

    
print('\n')
print('---------------- Improved Recommendation with Entity Resolution ---------------------------------------------- \n')
matrix = user_artist_ner_matrix
list_similar_users_ner, top_artists_ner = recommend_top_artists(user_id, user_cosine_similarity(matrix), matrix,
                                    similar_users = similar_users, top_artist = top_artist
                                   )
print(f"Top {top_artist} NER artist recommendations for user {user_id} : \n {top_artists_ner} \n")
print(f"Here are the {similar_users} similar users \n \t {list_similar_users_ner}")
for u in list_similar_users_ner:
    print(f"\t Artist frequented by {u}: \n \
        {user_artist_ner_matrix.loc[u].sort_index().sort_values(ascending=False).head(top_artist).index.tolist()}")


Top 5 artist recommendations for user 5c2b840d66d02ac7aa6a069eddd17f48 : 
 ['spice girls', 'jay-z', 'jack johnson', 'lil wayne', 'pretty lights'] 

Here are the 5 similar users 
 	 ['7017180d62cb57488fc8bb058b3c3a8e', '4fe4a15f91a6ac37af77dfee99e11957', '01ad7fc79498ab698b3f0e0f2c5c818e', 'ac08669f9d162aea8a4c599f311ef979', 'abfe2c0cc529a4cf5513604c1e422e83']
	 Artist frequented by 7017180d62cb57488fc8bb058b3c3a8e: 
             ['spice girls', '+/-', 'ne-yo', 'naughty by nature', 'naughty boy'] 
	 Artist frequented by 4fe4a15f91a6ac37af77dfee99e11957: 
             ['fleetwood mac', 'spice girls', '+/-', 'ne-yo', 'naughty by nature'] 
	 Artist frequented by 01ad7fc79498ab698b3f0e0f2c5c818e: 
             ['pretty lights', 'avicii', 'lil wayne', 'glee cast', 'deadmau5'] 
	 Artist frequented by ac08669f9d162aea8a4c599f311ef979: 
             ['mumford & sons', '+/-', 'natalie imbruglia', 'ndidi o', 'naughty by nature'] 
	 Artist frequented by abfe2c0cc529a4cf5513604c1e422e83: 
         

### get top artist recommendation for all users

In [61]:
user_top_artists_recommendation = {}

df_user_cosine_similarity = user_cosine_similarity(user_artist_matrix)

df_user_ner_cosine_similarity = user_cosine_similarity(user_artist_ner_matrix)

similar_users = 5
top_artist = 3
for user_id in df.user_id:
    user_top_artists_recommendation[user_id] = []
    list_similar_users, top_artists = recommend_top_artists(user_id, df_user_cosine_similarity, user_artist_matrix,
                                    similar_users = similar_users, top_artist = top_artist 
                                   )
    user_top_artists_recommendation[user_id].append(top_artists)
    
    # ner 
    list_similar_users, top_ner_artists = recommend_top_artists(user_id, df_user_ner_cosine_similarity, user_artist_ner_matrix,
                                similar_users = similar_users, top_artist = top_artist 
                               )
    user_top_artists_recommendation[user_id].append(top_ner_artists)

In [66]:
df_top_artist_recommendation = pd.DataFrame.from_dict(user_top_artists_recommendation,orient='columns').transpose()
df_top_artist_recommendation.columns=['top artist recommendation','Improved - entity Resolution top artist recommendation']

df_top_artist_recommendation.head(50)

top artist recommendation  \
5c2b840d66d02ac7aa6a069eddd17f48                 [spice girls, jay-z, jack johnson]   
6da651f45aac18e5da52433aa3c38947             [flying lotus, miles davis, minutemen]   
20350023c94899ce25cd1a86ea385853  [robbie williams, english chamber orchestra, c...   
f9809d1afbab1b0aaa0c2f678a90752a   [arcade fire, nine inch nails, john butler trio]   
db9646b77f3a677040b9140489da16f1                     [bastille, +/-, natalie grant]   
7ee6797934d627b54a72dda785b1ea85         [dinosaur jr, jack white, jerry goldsmith]   
5c4438bac13e50d6db5058ad0b0983da               [joni mitchell, jamie cullum, alt-j]   
705e46c3ff12a665e14ab67c497f18ac  [rolling stones, ella fitzgerald, mannheim ste...   
ec4faa478b43b87aa990f5f2b7d0f414               [nine inch nails, beck, frank zappa]   
500e8b67136775d227d35a1bb37478aa              [awolnation, darkside, clint mansell]   
d993ff8f2de226e2c6803e47a22e9d7e  [frank sinatra, robin trower, english chamber ...   
d087387cd3a21a7df6148facf7e6de3b     [fat larrys band, ottmar liebert, jethro tull]   
4d3478132c2411da4f59caa8d05f9020      [queens of the stone age, alunageorge, kyuss]   
a1636d750f67c7e6754165a2f7856860          [notorious big, green day, natalie grant]   
39bd5bffec13caec0918b1477ab70614                    [depeche mode, +/-, neal casal]   
a51bf8359b92019872ae25be2edcfe5f                    [linkin park, editors, incubus]   
822626a6f453a5ed2eeb0d1b6686b714             [gorillaz, vicentico, bloodhound gang]   
92decd9488a6d8f7feb6117f6cb8a17a  [robbie williams, english chamber orchestra, c...   
05d9f2d60012d07349cb41690d29bba6       [frank sinatra, sufjan stevens, jethro tull]   
9f73a2dece773a105003273316cff35d                     [blur, +/-, natalie imbruglia]   
4398de6902abde3351347b048fcdc287    [anberlin, lana del rey, further seems forever]   
41a882966229719963b528a6f34e5c56            [green day, geto boys, suicide silence]   
ea0e4bd5eb86a066c6ff6a275c253704    [robbie williams, hall√© orchestra, little feat]   
490df4a34d7b1e0c7b82e68b66d23814                [weezer, vicetone, fat larrys band]   
00ba6890ed516e47967114c2f75bbe35  [marilyn manson, rolling stones, ella fitzgerald]   
e6f92f64f6bc13be7d4353a0135429f9                [crystal castles, washed out, beck]   
c027b267ac4ae0fd1490890b838c4235                 [alejandro sanz, fito p√°ez, guaco]   
dd989ee07bd90c22cfcddbd35e446d7f                [simply red, mecano, john coltrane]   
04bfc6d99a244c7df3bce93cc6e23c3d                   [hives, slowdive, 3 colours red]   
9ccb672668dd60a1a8eb72303476687e                [bud powell, black crowes, nirvana]   
3d70271aad91068c4a577d426e2aa19e  [andrew bird, robbie williams, english chamber...   
c8d8a7b5590bf08bb351757686304152  [kool & the gang, english chamber orchestra, c...   
08280e52f83cce4aa0f4ba12ac09d01b                         [david bowie, smiths, +/-]   
00ad223995792793c9ee902cd545ef7b        [robbie williams, human beinz, melt banana]   
c67f3a551b435a04fadfe46fbda1c3a5                 [alanis morissette, weezer, queen]   
9f91e8dc7f5543ba408e437fe4ede243  [robbie williams, english chamber orchestra, c...   
144ced6d2b5322c74e4a2efa9ea5e69b  [hiss golden messenger, lights & motion, story...   
1f279c39d9bb0a4c5b7a0653b44a8017                      [kooks, scorpions, jovanotti]   
5b82c20e6b45a79c382e80e96e04b956         [frank sinatra, robin trower, jethro tull]   
141970246646c7a0a6af2e1f6e834233                            [jay z, radiohead, +/-]   
9d90242856d051f07229bd60b0b7976c        [robbie williams, mat kearney, jethro tull]   
0a94d0cf8631c721c2c0c9b91fcd146e                   [s club 7, cypress hill, weezer]   
7f5e926e114fbb96e362a5ee581d6711  [kodaline, hillsong young & free, george fride...   
fd16fd12aab79c33ffd1944d6b2b9af8                   [human beinz, cornelius, weezer]   
c1c5244249d8d54099d75cda6e6c8a82     [robbie williams, elvis presley, robin trower]   
64c4686c595c76e0ff98743f1fab7439            [britney s

In [69]:
print('Total Users' , len(df_top_artist_recommendation.index))

per_users_artist_recom_impacted = round(len(
                                    np.where(
                                        (df_top_artist_recommendation['top artist recommendation'] != df_top_artist_recommendation['Improved - entity Resolution top artist recommendation'])
                                    )[0])*100/len(df_top_artist_recommendation.index),2)

print()
print('--------------------------------------------------------------------------- \n ')
print(f"\t {per_users_artist_recom_impacted}% user's recommendation improved with Entity Resolution")
print('\n---------------------------------------------------------------------------')

Total Users 5009

--------------------------------------------------------------------------- 
 
	 82.93% user's recommendation improved with Entity Resolution

--------------------------------------------------------------------------


# -------------------------------------------------------------------------------------

### Next steps or challenges:
#### - Limited data. Data Collection like 
    1. Content data 
        song metadata such as release year and duration
        attributes of songs such as tempo, key, loudness, modesuch as tempo, genre, mood
    
    2. User data
        interaction data: skips, likes, dislikes, shares, and playlist additions.
        search queries: search terms users use to find music.
##### - Working with larger volume of data. 
##### - Editorial input in artist name - Named Entity Resolution would further improve this feature thereby model's performance